In [3]:
import nbimporter
import dataDownloader as dd

Importing Jupyter notebook from dataDownloader.ipynb


In [4]:
def getPreviousMatches(matches,x,teamVenue,predictionMatches):
    previousMatches = []
    for i in range(x-1,0,-1):
        if(len(previousMatches) == predictionMatches):
            break
        if(matches[x][teamVenue]==matches[i]['HomeTeam'] or matches[x][teamVenue]==matches[i]['AwayTeam']):
            previousMatches.append(matches[i])
    return previousMatches

In [35]:
import glob
import csv
import random

def generateLearningSet(pastMatchesTaken):
    #Read files
    filesPaths = glob.glob("../data/*.csv")
    x_data = []
    y_data = []
    for dataFile in filesPaths: 
        print(f"Reading file {dataFile}")
        with open(dataFile) as csvfile:
            reader = csv.DictReader(csvfile)
            #print(f"reading file {dataFile} with {sum(1 for line in reader)} entries")
            matches = []
            try:
                for row in reader:
                    matches.append(row)
                for x in range(len(matches)):
                    homePreviousMatches = getPreviousMatches(matches, x, 'HomeTeam', pastMatchesTaken)
                    awayPreviousMatches = getPreviousMatches(matches, x,'AwayTeam', pastMatchesTaken)
                    temp = []
                    if len(homePreviousMatches) == pastMatchesTaken and len(awayPreviousMatches) == pastMatchesTaken:
                        #print(f"{x}: {matches[x]['HomeTeam']} vs {matches[x]['AwayTeam']}")

                        for hm in homePreviousMatches:
                            # temp = []
                            if matches[x]['HomeTeam'] == hm['HomeTeam']:
                                temp.append(int(float(hm['FTHG']))-int(float(hm['FTAG'])))
                                temp.append(int(float(hm['HS'])))
                                temp.append(int(float(hm['AS'])))
                                temp.append(int(float(hm['HST'])))
                                temp.append(int(float(hm['AST'])))
                                temp.append(int(float(hm['HC'])))
                                temp.append(int(float(hm['AC'])))
                            else:
                                temp.append(int(float(hm['FTAG']))-int(float(hm['FTHG'])))
                                temp.append(int(float(hm['AS'])))
                                temp.append(int(float(hm['HS'])))
                                temp.append(int(float(hm['AST'])))
                                temp.append(int(float(hm['HST'])))
                                temp.append(int(float(hm['AC'])))
                                temp.append(int(float(hm['HC'])))
                            #matchLearningData.append(temp)

                        for am in homePreviousMatches:
                            # temp = []
                            if matches[x]['AwayTeam'] == am['HomeTeam']:
                                temp.append(int(float(am['FTHG']))-int(float(am['FTAG'])))
                                temp.append(int(float(am['HS'])))
                                temp.append(int(float(am['AS'])))
                                temp.append(int(float(am['HST'])))
                                temp.append(int(float(am['AST'])))
                                temp.append(int(float(am['HC'])))
                                temp.append(int(float(am['AC'])))
                            else:
                                temp.append(int(float(am['FTAG']))-int(float(am['FTHG'])))
                                temp.append(int(float(am['AS'])))
                                temp.append(int(float(am['HS'])))
                                temp.append(int(float(am['AST'])))
                                temp.append(int(float(am['HST'])))
                                temp.append(int(float(am['AC'])))
                                temp.append(int(float(am['HC'])))
                            #matchLearningData.append(temp)
                            

                        x_data.append(temp)
                        if(int(float(matches[x]['FTHG'])) > int(float(matches[x]['FTAG']))):
                            y_data.append(1)
                        elif (int(float(matches[x]['FTAG'])) > int(float(matches[x]['FTHG']))):
                            y_data.append(-1)
                        else:
                            y_data.append(0)
            except Exception as e:
                print(f"Problem with league {dataFile} : {e}")
                pass
                    
    return x_data, y_data

def createTrainAndTestSets(x_data, y_data, trainToOverallRatio):
    x_train = []
    y_train = []
    x_test = []
    y_test = []
    for i in range(len(x_data)):
        if(random.random() < trainToOverallRatio):
            x_train.append(x_data[i])
            y_train.append(y_data[i])
        else:
            x_test.append(x_data[i])
            y_test.append(y_data[i])
    return x_train, y_train, x_test, y_test

In [49]:
x_data, y_data = generateLearningSet(10)
x_train, y_train, x_test, y_test = createTrainAndTestSets(x_data, y_data, 0.7)

print(f"overall matches count: {len(x_data)}")
print(f"train matches count: {len(x_train)}   {len(x_train)/len(x_data)}% of overall")
print(f"test matches count: {len(x_test)}")
print(x_train[0])
print(y_train[0])

Reading file ../data\B1-1314.csv
Problem with league ../data\B1-1314.csv : 'AS'
Reading file ../data\B1-1415.csv
Problem with league ../data\B1-1415.csv : 'AS'
Reading file ../data\B1-1516.csv
Problem with league ../data\B1-1516.csv : 'AS'
Reading file ../data\B1-1617.csv
Problem with league ../data\B1-1617.csv : 'AS'
Reading file ../data\B1-1718.csv
Reading file ../data\B1-1819.csv
Reading file ../data\B1-1920.csv
Reading file ../data\D1-1314.csv
Reading file ../data\D1-1415.csv
Reading file ../data\D1-1516.csv
Reading file ../data\D1-1617.csv
Reading file ../data\D1-1718.csv
Reading file ../data\D1-1819.csv
Reading file ../data\D1-1920.csv
Reading file ../data\D2-1314.csv
Problem with league ../data\D2-1314.csv : 'AS'
Reading file ../data\D2-1415.csv
Problem with league ../data\D2-1415.csv : 'AS'
Reading file ../data\D2-1516.csv
Problem with league ../data\D2-1516.csv : 'AS'
Reading file ../data\D2-1617.csv
Problem with league ../data\D2-1617.csv : 'AS'
Reading file ../data\D2-1718.c

In [45]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

def fit(x_train, y_train):
    clf.fit(x_train,y_train)
    print(clf.class_count_)

def predict(x_test):
    return clf.predict(x_test)

def score(X,y):
    return clf.score(X,y)

In [50]:
fit(x_train, y_train)
predictions = predict(x_test)
print(score(x_test,y_test))
# print(predictions)
# print(y_test)

[5553. 4777. 8265.]
0.4268308160934129
